<h3> Części mowy (POS)

In [1]:
text = "The Very Large Telescope (VLT) of the European Southern Observatory (ESO), an array of four individual telescopes in the Atacama desert, has given us a huge amount of new data about the universe. Researchers have now used it to find a group of six galaxies around a supermassive black hole, from when the Universe was just 0.9 billion years old - it's estimated to be 13.8 billion years old now. Black holes are thought to sit at the center of galaxies including the Milky Way. "

In [2]:
from nltk import sent_tokenize, word_tokenize

In [3]:
sentences = sent_tokenize(text.lower()) #podzial na sentencje 
print(sentences)

['the very large telescope (vlt) of the european southern observatory (eso), an array of four individual telescopes in the atacama desert, has given us a huge amount of new data about the universe.', "researchers have now used it to find a group of six galaxies around a supermassive black hole, from when the universe was just 0.9 billion years old - it's estimated to be 13.8 billion years old now.", 'black holes are thought to sit at the center of galaxies including the milky way.']


In [4]:
tokens_sen1 = word_tokenize(sentences[0]) #podzial na slowa pierwszego zdania
print(tokens_sen1)

['the', 'very', 'large', 'telescope', '(', 'vlt', ')', 'of', 'the', 'european', 'southern', 'observatory', '(', 'eso', ')', ',', 'an', 'array', 'of', 'four', 'individual', 'telescopes', 'in', 'the', 'atacama', 'desert', ',', 'has', 'given', 'us', 'a', 'huge', 'amount', 'of', 'new', 'data', 'about', 'the', 'universe', '.']


In [7]:
from nltk.probability import FreqDist

fdist = FreqDist(tokens_sen1)
fdist

FreqDist({'the': 4, 'of': 3, '(': 2, ')': 2, ',': 2, 'very': 1, 'large': 1, 'telescope': 1, 'vlt': 1, 'european': 1, ...})

In [8]:
fdist.most_common(2) #dwa najczęstsze

[('the', 4), ('of', 3)]

In [9]:
from nltk import pos_tag #Części mowy

tags = pos_tag(tokens_sen1)
print(tags)

[('the', 'DT'), ('very', 'RB'), ('large', 'JJ'), ('telescope', 'NN'), ('(', '('), ('vlt', 'NN'), (')', ')'), ('of', 'IN'), ('the', 'DT'), ('european', 'JJ'), ('southern', 'JJ'), ('observatory', 'NN'), ('(', '('), ('eso', 'NN'), (')', ')'), (',', ','), ('an', 'DT'), ('array', 'NN'), ('of', 'IN'), ('four', 'CD'), ('individual', 'JJ'), ('telescopes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('atacama', 'NN'), ('desert', 'NN'), (',', ','), ('has', 'VBZ'), ('given', 'VBN'), ('us', 'PRP'), ('a', 'DT'), ('huge', 'JJ'), ('amount', 'NN'), ('of', 'IN'), ('new', 'JJ'), ('data', 'NNS'), ('about', 'IN'), ('the', 'DT'), ('universe', 'NN'), ('.', '.')]


<h4> Słowa nieistniejące 

In [10]:
tags2 = pos_tag(['gutvidual']) #a jednak przewiduje... :)
tags2

[('gutvidual', 'JJ')]

<h4> Słowa w zależności od kontekstu mogą mieć różne POS tagi

In [11]:
print(pos_tag(word_tokenize("I cannot bear this headache"))) #bear jest czasownikiem

[('I', 'PRP'), ('can', 'MD'), ('not', 'RB'), ('bear', 'VB'), ('this', 'DT'), ('headache', 'NN')]


In [12]:
print(pos_tag(word_tokenize("Yesterday, I saw a bear")))  #bear jest rzeczownikiem

[('Yesterday', 'NN'), (',', ','), ('I', 'PRP'), ('saw', 'VBD'), ('a', 'DT'), ('bear', 'NN')]


<h4> Uczenie HMM - jaki POS?

Problem:

- dana jest sekwencja wyrazów  $x_1x_2x_3...x_n$  (przykładowo ["I", "like", "dogs"])
- cel: znaleźć najbardziej prawdopodobną sekwencje tagów  $y_1y_2y_3...y_n$  (przykładowo ["PRP","VBP", "NNS"]), czyli:

$argmax_{y_1,y_2,y_3...,y_n}P(x_1,x_2,x_3,...,x_n,y_1,y_2,y_3,...,y_n)$

- Jak? 

Algorytm Viterbiego

- Stany:

obserwacje = słowa

stany ukryte = tagi POS


- Uczenie algorytmu polega na wyznaczeniu:

prawdopodobieństw przejść = prawdopodobieństwa typu P(VP|NP)

prawdopodobieńsw emisji = prawdopodobieństwa typu P(John|VP)

In [16]:
from nltk.corpus import treebank  #korpus do testowania algorytmow przewidujących częsci mowy

treebank.readme().replace('\n', ' ')[:260]

'[ PENN TREEBANK SAMPLE ]\r http://www.cis.upenn.edu/~treebank/home.html\r \r This is a ~5% fragment of Penn Treebank, (C) LDC 1995.  It is made\r available under fair use for the purposes of illustrating NLTK tools\r for tokenizing, tagging, chunking and parsing.  '

In [18]:
print(treebank.tagged_sents()[:2]) #pierwsze dwie otagowane sentencje

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')]]


In [19]:
len(treebank.tagged_sents()) #ile wszystkich otagowanych sentencji?

3914

<h4> HMM

In [20]:
from nltk.tag import hmm  #Ukryte modele Markowa

trainer = hmm.HiddenMarkovModelTrainer() #buduje model HMMM
tagger = trainer.train_supervised(treebank.tagged_sents()) #trenuje model (metoda największej wiarygodności) na wszystkich otagowanych sentencjach z treebank

In [21]:
tagger.tag(['today','is','such','a','very','nice','day']) #patrze jak model sprawdza sie na przykladowych danych

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'PDT'),
 ('a', 'DT'),
 ('very', 'RB'),
 ('nice', 'JJ'),
 ('day', 'NN')]

In [22]:
tagger.tag(['today','is','such','a','picturesque','day']) #patrze jak model sprawdza sie na innych przykladowych danych

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'PDT'),
 ('a', 'DT'),
 ('picturesque', 'NNP'),
 ('day', 'NNP')]

In [23]:
pos_tag(['today','is','such','a','picturesque','day']) #POS tagging przy uzyciu metody pos_tag, dla porownania

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'JJ'),
 ('a', 'DT'),
 ('picturesque', 'JJ'),
 ('day', 'NN')]

In [24]:
tagger.evaluate(treebank.tagged_sents()) #jaka jest przewidywalnosc modelu na całym korpusie treebank?

0.9815546902936152

<h4> Zadanie1: Podziel wyjściowy zbiór sentencji $treebank.tagged\_sents()$ na dwa zbiory: pierwsze 3000 sentencje zapisz pod zmienną  $trained$, natomiast pozostałe sentencje zapisz pod zmienną $tested$.Zbuduj model tylko w oparciu o $trained$. Następnie wykonaj ewaluacje modelu zarówno na  $trained$  jak i  $tested$. Zastanów się z czego mogą wynikać rozbieżności.

<h3> Porcjowanie (Chunking)


In [28]:
print(tags)

[('the', 'DT'), ('very', 'RB'), ('large', 'JJ'), ('telescope', 'NN'), ('(', '('), ('vlt', 'NN'), (')', ')'), ('of', 'IN'), ('the', 'DT'), ('european', 'JJ'), ('southern', 'JJ'), ('observatory', 'NN'), ('(', '('), ('eso', 'NN'), (')', ')'), (',', ','), ('an', 'DT'), ('array', 'NN'), ('of', 'IN'), ('four', 'CD'), ('individual', 'JJ'), ('telescopes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('atacama', 'NN'), ('desert', 'NN'), (',', ','), ('has', 'VBZ'), ('given', 'VBN'), ('us', 'PRP'), ('a', 'DT'), ('huge', 'JJ'), ('amount', 'NN'), ('of', 'IN'), ('new', 'JJ'), ('data', 'NNS'), ('about', 'IN'), ('the', 'DT'), ('universe', 'NN'), ('.', '.')]


In [25]:
from nltk.chunk import RegexpParser

grammar = "chunk: {<DT>?<JJ>*<NN>}" #DT - determiner/określnik, JJ - adjective/przymiotnik, NN - noun/rzeczonik
chunker = RegexpParser(grammar) 
result = chunker.parse(tags) #tags zdefiniowane bylo wyzej

In [26]:
print(result)

(S
  the/DT
  very/RB
  (chunk large/JJ telescope/NN)
  (/(
  (chunk vlt/NN)
  )/)
  of/IN
  (chunk the/DT european/JJ southern/JJ observatory/NN)
  (/(
  (chunk eso/NN)
  )/)
  ,/,
  (chunk an/DT array/NN)
  of/IN
  four/CD
  individual/JJ
  telescopes/NNS
  in/IN
  (chunk the/DT atacama/NN)
  (chunk desert/NN)
  ,/,
  has/VBZ
  given/VBN
  us/PRP
  (chunk a/DT huge/JJ amount/NN)
  of/IN
  new/JJ
  data/NNS
  about/IN
  (chunk the/DT universe/NN)
  ./.)


In [27]:
print(result[0]) #mozna sie odwolywac

('the', 'DT')


<h4> Zadanie2: Zapisz do listy wszystkie chunki występujące w tekście a spełniajacy zadany (przez siebie) warunek.

In [87]:
result.draw() #rysowanie tego co wyzej (pojawia sie nowe okno)

<h4> Zadanie3: Zinterpretuj poniższy chunk.

In [30]:
grammar = "chunk: {<NN.?>*<VBD.?>*<JJ.?>*<CC>?}" #co oznacza wyrazenie regularne zapisane w {}?
chunker = RegexpParser(grammar) 
result = chunker.parse(tags) #tags zdefiniowane bylo wyzej
result.draw()

<h4> Zadanie4: Pobierz dowolny tekst (przez f = open(...)). Następnie wyznacz dla niego zaproponowany przez siebie chunk.

<h3> Rozpoznawanie bytów (Entity Recognition)


In [31]:
tokens_sen2 = word_tokenize("Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879.")
tags2 = pos_tag(tokens_sen2)
print(tags2)

[('Albert', 'NNP'), ('Einstein', 'NNP'), ('was', 'VBD'), ('born', 'VBN'), ('at', 'IN'), ('Ulm', 'NNP'), (',', ','), ('in', 'IN'), ('Württemberg', 'NNP'), (',', ','), ('Germany', 'NNP'), (',', ','), ('on', 'IN'), ('March', 'NNP'), ('14', 'CD'), (',', ','), ('1879', 'CD'), ('.', '.')]


In [36]:
from nltk import ne_chunk

namedEnt = ne_chunk(tags2, binary = True) #przeszukiwanie pod kątem osób, miejsc itd...
print(namedEnt)

(S
  (NE Albert/NNP Einstein/NNP)
  was/VBD
  born/VBN
  at/IN
  (NE Ulm/NNP)
  ,/,
  in/IN
  (NE Württemberg/NNP)
  ,/,
  (NE Germany/NNP)
  ,/,
  on/IN
  March/NNP
  14/CD
  ,/,
  1879/CD
  ./.)


In [86]:
namedEnt.draw()

<h4> Zadanie5: Znajdz wszystkie byty (entity) w tekście poniżej.

In [38]:
txt = "Google, LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware. It is considered one of the Big Five technology companies in the U.S. information technology industry, alongside Amazon, Facebook, Apple, and Microsoft."

<h3> Stemowanie (Stemming)

In [39]:
from nltk.stem import PorterStemmer

In [40]:
ps = PorterStemmer() #Tworze stemer

words = ["wait", "waiting", "waited", "waits"]

for elem in words:
    print(ps.stem(elem)) #stemming

wait
wait
wait
wait


<h4> Zadanie6: Napisz funkcję do Stemmingu zdań. 


Wskazówka: Podziel tekst na tokeny, potem dokonaj stemmingu na każdym tokenie, zapisz wyrazy do listy, na końcu z użyciem join połącz wszystko w jeden ciągły tekst.

<h3> Lematyzacja (Lemmatization) - "lepszy Stemming"

In [42]:
from nltk.stem import WordNetLemmatizer

In [45]:
lemmatizer = WordNetLemmatizer() #tworze Lemmatyzator 
print(lemmatizer.lemmatize("cats")) #wywołuje metode lemmatize z tego Lemmatyzatora na słowie 'cat'

cat


In [46]:
print(lemmatizer.lemmatize("feet"))

foot


In [47]:
print(lemmatizer.lemmatize("better"))

better


In [48]:
print(lemmatizer.lemmatize("better", pos = 'r')) #n-noun/rzeczownik (domyslnie), a-adjective/przymiotnik, v-verb/czasownik, r-adverb/przyslowek

well


<h4> Zadanie7: Napisz funkcję do Lemmatyzacji zdań. W tym przypadku użyteczne mogą być tagi POS:
        
- zaczynające się od J uznaj za przymiotniki
- zaczynające się od N to rzeczowniki
- zaczynające się od V do czasowniki
- zaczynające się od R to pzysłówki

<h3> Sieć słów (Wordnet)

In [49]:
from nltk.corpus import wordnet as wn

In [50]:
wn.synsets("automobile") #jakie są synstety (zbiory synonimow) dla slowa automobile?

[Synset('car.n.01'), Synset('automobile.v.01')]

In [51]:
wn.synset('car.n.01').lemma_names() #jakie są słowa w synsecie car.n.01; to synset obejmujący różne synonimy (lemmas)

['car', 'auto', 'automobile', 'machine', 'motorcar']

In [52]:
wn.synset('car.n.01').definition() #jaka jest definicja tego synsetu?

'a motor vehicle with four wheels; usually propelled by an internal combustion engine'

In [53]:
wn.synset('car.n.01').examples() #przykladowe uzycie w zdaniu

['he needs a car to get to work']

In [54]:
wn.synsets("car") #jakie są synsety slowa car?

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

In [55]:
#jak wyglada te 5 synsetow?

for synset in wn.synsets("car"):
    print(synset.lemma_names())

['car', 'auto', 'automobile', 'machine', 'motorcar']
['car', 'railcar', 'railway_car', 'railroad_car']
['car', 'gondola']
['car', 'elevator_car']
['cable_car', 'car']


<h3> Hiponimy i hiperonimy

In [56]:
motorcar = wn.synset("car.n.01")

In [57]:
types_of_cars = motorcar.hyponyms() #hiponim to takie slowo ktorego znaczenie semanatyczne pochodzi od innego slowa (hiperonimu) np rozowy to kolor, gęś to ptak
print(types_of_cars[14]) #przykład hiponimu (synsetu zawierającego hiponimy)

Synset('jeep.n.01')


In [58]:
wn.synset('jeep.n.01').lemma_names() #slowa w synsecie jeep.n.01

['jeep', 'landrover']

In [59]:
#hiponimy dla slowa car (z synsetu car.n.01)
print([lemma.name() for synset in types_of_cars for lemma in synset.lemmas()])

['ambulance', 'beach_wagon', 'station_wagon', 'wagon', 'estate_car', 'beach_waggon', 'station_waggon', 'waggon', 'bus', 'jalopy', 'heap', 'cab', 'hack', 'taxi', 'taxicab', 'compact', 'compact_car', 'convertible', 'coupe', 'cruiser', 'police_cruiser', 'patrol_car', 'police_car', 'prowl_car', 'squad_car', 'electric', 'electric_automobile', 'electric_car', 'gas_guzzler', 'hardtop', 'hatchback', 'horseless_carriage', 'hot_rod', 'hot-rod', 'jeep', 'landrover', 'limousine', 'limo', 'loaner', 'minicar', 'minivan', 'Model_T', 'pace_car', 'racer', 'race_car', 'racing_car', 'roadster', 'runabout', 'two-seater', 'sedan', 'saloon', 'sport_utility', 'sport_utility_vehicle', 'S.U.V.', 'SUV', 'sports_car', 'sport_car', 'Stanley_Steamer', 'stock_car', 'subcompact', 'subcompact_car', 'touring_car', 'phaeton', 'tourer', 'used-car', 'secondhand_car']


In [60]:
vehicles = motorcar.hypernyms() #hiperonimy (synsety) dla car.n.01
print(vehicles)

[Synset('motor_vehicle.n.01')]


In [61]:
print(sorted([lemma.name() for synset in vehicles for lemma in synset.lemmas()])) #wszystkie hiperonimy

['automotive_vehicle', 'motor_vehicle']


<h4> Zadanie8: Znajdź wszystkie hiponimy i hiperonimy dla słowa $dog$. Zacznij od wyszukania synsetów dla tego słowa, wybierz pierwszy z nich.

<h3> Synonimy

In [62]:
synonyms = []
for syn in wn.synsets('girl'):
    for lemma in syn.lemmas(): 
        synonyms.append(lemma.name())

print(synonyms)

['girl', 'miss', 'missy', 'young_lady', 'young_woman', 'fille', 'female_child', 'girl', 'little_girl', 'daughter', 'girl', 'girlfriend', 'girl', 'lady_friend', 'girl']


<h4> Antonimy

In [63]:
antonyms = []
for syn in wn.synsets("girl"):
    for l in syn.lemmas():
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(antonyms)

['male_child', 'boy', 'son', 'boy']


<h4> Zadanie9: Znajdź synonimy i antonimy słowa $happy$.

<h3> Metryka Wu-Palmera - podobieństwo synsetów


In [64]:
bike = wn.synsets('bicycle')[0]
boy = wn.synsets('boy')[0]
human = wn.synsets('human')[0]

print(bike)
print(boy)
print(human)

Synset('bicycle.n.01')
Synset('male_child.n.01')
Synset('homo.n.02')


In [65]:
wn.synset('homo.n.02').lemma_names() #zeby srpawdzic co tam jest

['homo', 'man', 'human_being', 'human']

In [66]:
bike.wup_similarity(human), boy.wup_similarity(human), boy.wup_similarity(bike) #licze na ile slowa (a formalnie synsety są podobne do siebie)

(0.34782608695652173, 0.5217391304347826, 0.4444444444444444)

<h4> Zadanie10: Porównaj podobieństwo słów dog, cat i fish.

<h3> Uzupełnienie: N-gramy

In [67]:
from nltk import ngrams

In [68]:
ex_text = "This is an example paper in which I summed up some basic concepts reffering to viruses."

<h4> N-gramy dla słów

In [69]:
print(list(ngrams(word_tokenize(ex_text),2)))

[('This', 'is'), ('is', 'an'), ('an', 'example'), ('example', 'paper'), ('paper', 'in'), ('in', 'which'), ('which', 'I'), ('I', 'summed'), ('summed', 'up'), ('up', 'some'), ('some', 'basic'), ('basic', 'concepts'), ('concepts', 'reffering'), ('reffering', 'to'), ('to', 'viruses'), ('viruses', '.')]


In [70]:
print(list(ngrams(word_tokenize(ex_text),3)))

[('This', 'is', 'an'), ('is', 'an', 'example'), ('an', 'example', 'paper'), ('example', 'paper', 'in'), ('paper', 'in', 'which'), ('in', 'which', 'I'), ('which', 'I', 'summed'), ('I', 'summed', 'up'), ('summed', 'up', 'some'), ('up', 'some', 'basic'), ('some', 'basic', 'concepts'), ('basic', 'concepts', 'reffering'), ('concepts', 'reffering', 'to'), ('reffering', 'to', 'viruses'), ('to', 'viruses', '.')]


Przykład: Zliczanie występowania n-gramów na przykładzie przemówienia Trumpa z 2017.

In [76]:
from nltk.corpus import inaugural
from nltk.corpus import stopwords
import string

stop_words =  set(stopwords.words("english"))
Trump = [w.lower() for w in inaugural.words('2017-Trump.txt') if w not in stop_words and w not in string.punctuation] 

In [77]:
len(list(ngrams(Trump,3))),len(set(ngrams(Trump,3)))

(791, 782)

In [78]:
from collections import Counter #Klasa do zliczania

counts = Counter(list(ngrams(Trump,3)))
print(counts.most_common(10)) #10 najpopularniejszych

[(('we', 'bring', 'back'), 3), (('we', 'make', 'america'), 3), (('united', 'states', 'america'), 2), (('from', 'day', 'forward'), 2), (('together', 'make', 'america'), 2), (('thank', 'god', 'bless'), 2), (('god', 'bless', 'america'), 2), (('chief', 'justice', 'roberts'), 1), (('justice', 'roberts', 'president'), 1), (('roberts', 'president', 'carter'), 1)]


<h4> N-gramy dla znaków

In [79]:
generated_4grams = []

for word in word_tokenize(ex_text):
    generated_4grams.append(list(ngrams(word, 4, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))

In [80]:
print(generated_4grams[:5])

[[('_', '_', '_', 'T'), ('_', '_', 'T', 'h'), ('_', 'T', 'h', 'i'), ('T', 'h', 'i', 's'), ('h', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_')], [('_', '_', '_', 'i'), ('_', '_', 'i', 's'), ('_', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_')], [('_', '_', '_', 'a'), ('_', '_', 'a', 'n'), ('_', 'a', 'n', '_'), ('a', 'n', '_', '_'), ('n', '_', '_', '_')], [('_', '_', '_', 'e'), ('_', '_', 'e', 'x'), ('_', 'e', 'x', 'a'), ('e', 'x', 'a', 'm'), ('x', 'a', 'm', 'p'), ('a', 'm', 'p', 'l'), ('m', 'p', 'l', 'e'), ('p', 'l', 'e', '_'), ('l', 'e', '_', '_'), ('e', '_', '_', '_')], [('_', '_', '_', 'p'), ('_', '_', 'p', 'a'), ('_', 'p', 'a', 'p'), ('p', 'a', 'p', 'e'), ('a', 'p', 'e', 'r'), ('p', 'e', 'r', '_'), ('e', 'r', '_', '_'), ('r', '_', '_', '_')]]


In [81]:
generated_4grams = [word for sublist in generated_4grams for word in sublist] #żeby nie bylo listy list jak wyzej

print(generated_4grams[:10])

[('_', '_', '_', 'T'), ('_', '_', 'T', 'h'), ('_', 'T', 'h', 'i'), ('T', 'h', 'i', 's'), ('h', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_'), ('_', '_', '_', 'i'), ('_', '_', 'i', 's'), ('_', 'i', 's', '_')]


In [82]:
ng_list_4grams = generated_4grams
for idx, val in enumerate(generated_4grams):
    ng_list_4grams[idx] = ''.join(val)
    
print(ng_list_4grams)

['___T', '__Th', '_Thi', 'This', 'his_', 'is__', 's___', '___i', '__is', '_is_', 'is__', 's___', '___a', '__an', '_an_', 'an__', 'n___', '___e', '__ex', '_exa', 'exam', 'xamp', 'ampl', 'mple', 'ple_', 'le__', 'e___', '___p', '__pa', '_pap', 'pape', 'aper', 'per_', 'er__', 'r___', '___i', '__in', '_in_', 'in__', 'n___', '___w', '__wh', '_whi', 'whic', 'hich', 'ich_', 'ch__', 'h___', '___I', '__I_', '_I__', 'I___', '___s', '__su', '_sum', 'summ', 'umme', 'mmed', 'med_', 'ed__', 'd___', '___u', '__up', '_up_', 'up__', 'p___', '___s', '__so', '_som', 'some', 'ome_', 'me__', 'e___', '___b', '__ba', '_bas', 'basi', 'asic', 'sic_', 'ic__', 'c___', '___c', '__co', '_con', 'conc', 'once', 'ncep', 'cept', 'epts', 'pts_', 'ts__', 's___', '___r', '__re', '_ref', 'reff', 'effe', 'ffer', 'feri', 'erin', 'ring', 'ing_', 'ng__', 'g___', '___t', '__to', '_to_', 'to__', 'o___', '___v', '__vi', '_vir', 'viru', 'irus', 'ruse', 'uses', 'ses_', 'es__', 's___', '___.', '__._', '_.__', '.___']


<h4> Zadanie11: Napisz funkcję która dla zadanego tekstu zwraca słownik zliczeń N-gramów dla znaków (czyli funkcja ma dwa arugmenty: text i N). Przykładowe wywołanie dla N=4 i powyższego tekstu powinno zwrócić:

In [84]:
#

{'___T': 1, '__Th': 1, '_Thi': 1, 'This': 1, 'his_': 1, 'is__': 2, 's___': 4, '___i': 2, '__is': 1, '_is_': 1, '___a': 1, '__an': 1, '_an_': 1, 'an__': 1, 'n___': 2, '___e': 1, '__ex': 1, '_exa': 1, 'exam': 1, 'xamp': 1, 'ampl': 1, 'mple': 1, 'ple_': 1, 'le__': 1, 'e___': 2, '___p': 1, '__pa': 1, '_pap': 1, 'pape': 1, 'aper': 1, 'per_': 1, 'er__': 1, 'r___': 1, '__in': 1, '_in_': 1, 'in__': 1, '___w': 1, '__wh': 1, '_whi': 1, 'whic': 1, 'hich': 1, 'ich_': 1, 'ch__': 1, 'h___': 1, '___I': 1, '__I_': 1, '_I__': 1, 'I___': 1, '___s': 2, '__su': 1, '_sum': 1, 'summ': 1, 'umme': 1, 'mmed': 1, 'med_': 1, 'ed__': 1, 'd___': 1, '___u': 1, '__up': 1, '_up_': 1, 'up__': 1, 'p___': 1, '__so': 1, '_som': 1, 'some': 1, 'ome_': 1, 'me__': 1, '___b': 1, '__ba': 1, '_bas': 1, 'basi': 1, 'asic': 1, 'sic_': 1, 'ic__': 1, 'c___': 1, '___c': 1, '__co': 1, '_con': 1, 'conc': 1, 'once': 1, 'ncep': 1, 'cept': 1, 'epts': 1, 'pts_': 1, 'ts__': 1, '___r': 1, '__re': 1, '_ref': 1, 'reff': 1, 'effe': 1, 'ffer': 1

<h3> Uzupełnienie: Gramatyki

https://coling.epfl.ch/TP/corr/TP-parsing-sol.php

In [1]:
import nltk
from nltk.grammar import CFG
from nltk.parse.generate import generate
grammar = nltk.CFG.fromstring("""
S -> A W | B S
W -> A S | B W |
A -> 'a'
B -> 'b'
""")

list(generate(grammar, depth = 4)) #depth = maksymalna glębokosc drzewa

[['a', 'b'], ['a'], ['b', 'a']]

In [2]:
list(generate(grammar, depth = 5))

[['a', 'a', 'a'],
 ['a', 'b', 'b'],
 ['a', 'b'],
 ['a'],
 ['b', 'a', 'b'],
 ['b', 'a'],
 ['b', 'b', 'a']]

<h4> Pytanie: Jak powstało aaa?